In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

# 0. Load data

In [37]:
candidates_df = pd.read_pickle("data/candidates_df.p")
tweets_df = pd.read_pickle("data/tweets_df.p")

# 1. Scrape `go2senkyo.com` for candidate information

## 1.1. Assembly election candidates

In [3]:
a_url = "https://go2senkyo.com/local/senkyo/21754"
a_response = requests.get(a_url)
a_soup = BeautifulSoup(a_response.content, "html.parser")

In [45]:
a_vote_count = []
for candidate in a_soup.find_all("h2", class_="m_senkyo_result_data_ttl"):
    dict = {}
    katakana = candidate.find("a").find("span").text
    dict['name'] = candidate.find("a").text.strip("\n").strip("\t").replace(katakana, "")
    a_vote_count.append(dict)

In [54]:
candidate_num = 0
for vote_num in a_soup.find_all("td", class_="right"):
    vote_num_clean = re.findall("([0-9,.]+)\s票", vote_num.text)[0].replace(",", "")
    a_vote_count[candidate_num]['vote_count'] = vote_num_clean
    candidate_num += 1

In [55]:
for candidate in a_vote_count:
    row = candidates_df[candidates_df.name == candidate['name']]
    candidates_df.loc[row.index, "vote_count"] = candidate['vote_count']

## 1.2. Mayor election candidates

In [58]:
m_url = "https://go2senkyo.com/local/senkyo/21753"
m_response = requests.get(m_url)
m_soup = BeautifulSoup(m_response.content, "html.parser")

In [59]:
m_vote_count = []
for candidate in m_soup.find_all("h2", class_="m_senkyo_result_data_ttl"):
    dict = {}
    katakana = candidate.find("a").find("span").text
    dict['name'] = candidate.find("a").text.strip("\n").strip("\t").replace(katakana, "")
    m_vote_count.append(dict)

In [60]:
candidate_num = 0
for vote_num in m_soup.find_all("td", class_="right"):
    vote_num_clean = re.findall("([0-9,.]+)\s票", vote_num.text)[0].replace(",", "")
    m_vote_count[candidate_num]['vote_count'] = vote_num_clean
    candidate_num += 1

In [61]:
for candidate in m_vote_count:
    row = candidates_df[candidates_df.name == candidate['name']]
    candidates_df.loc[row.index, "vote_count"] = candidate['vote_count']

In [64]:
candidates_df.vote_count = candidates_df.vote_count.astype('float')

## 1.3. Create results column

In [69]:
results = np.where(
    (candidates_df.category == "A") & (candidates_df.vote_count >= 3006),
    1,
    np.where(
        (candidates_df.category == "M") & (candidates_df.vote_count == candidates_df[candidates_df.category == "M"].vote_count.max()),
        1,
        0
    ))

In [71]:
candidates_df['results'] = results

In [72]:
candidates_df.to_pickle("data/candidates_df.p")